In [1]:
import pyaudio
import librosa
import numpy as np
from numpy_ringbuffer import RingBuffer
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import pyaudio

import torch

import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torchvision

import numpy as np
import time
from  numpy_ringbuffer import RingBuffer
from torch.autograd import Variable
from threading import Thread
from time import sleep

In [2]:
#classes = ['HardRock', 'Indie', 'Metal', 'Pop', 'Punk']
#model  = models.resnet50(pretrained=True)
#model.load_state_dict(torch.load("./models/genresv1"));


classes = ['HardRock', 'Indie', 'Metal', 'Pop', 'Punk']
model  = models.resnet50(pretrained=True)
model.load_state_dict(torch.load("./models/genresv1"));

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
transform = transforms.Compose(
    [transforms.ToPILImage(),
    transforms.Resize(224),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
def audio_interfaces():
        """
        Extracts audio interfaces data

        :return list[AudioInterface]: Audio interfaces data
        """
        p = pyaudio.PyAudio()

        interfaces = []
        for i in range(p.get_device_count()):
            data = p.get_device_info_by_index(i)
            if 'hw' not in data['name']:
                interfaces.append(data)
        p.terminate()
        return interfaces
audio_interfaces()

[{'index': 0,
  'structVersion': 2,
  'name': 'MacBook Pro Microphone',
  'hostApi': 0,
  'maxInputChannels': 1,
  'maxOutputChannels': 0,
  'defaultLowInputLatency': 0.04852607709750567,
  'defaultLowOutputLatency': 0.01,
  'defaultHighInputLatency': 0.05868480725623583,
  'defaultHighOutputLatency': 0.1,
  'defaultSampleRate': 44100.0},
 {'index': 1,
  'structVersion': 2,
  'name': 'MacBook Pro Speakers',
  'hostApi': 0,
  'maxInputChannels': 0,
  'maxOutputChannels': 2,
  'defaultLowInputLatency': 0.01,
  'defaultLowOutputLatency': 0.010498866213151927,
  'defaultHighInputLatency': 0.1,
  'defaultHighOutputLatency': 0.020657596371882084,
  'defaultSampleRate': 44100.0}]

In [3]:
SampleRate = 22050
ringBuffer = RingBuffer(3 * SampleRate,np.float32)
pa = pyaudio.PyAudio()
running = True
MemoryInUse=False

In [4]:
def callback(in_data, frame_count, time_info, flag):
    global MemoryInUse
    global ringBuffer
    audio_data = np.frombuffer(in_data, dtype=np.float32)
    
    audio_data = librosa.resample(audio_data, 44100, SampleRate)
    while (MemoryInUse):
        pass
    MemoryInUse = True
    ringBuffer.extend(audio_data)
    MemoryInUse=False
    return None, pyaudio.paContinue

In [5]:
def infere_Class_Type():
    global ringBuffer
    global MemoryInUse
    global running
    n_fft = 1024
    hop_length = 256
    n_mels = 224
    fmin = 20
    fmax = SampleRate / 2
    while running:
        if(not ringBuffer.is_full):
            continue
        if(MemoryInUse):
            continue
        MemoryInUse=True
        audio_data = np.array(ringBuffer)
        MemoryInUse=False
        mel_spec_power = librosa.feature.melspectrogram(audio_data, sr=SampleRate, n_fft=n_fft, 
                                                    hop_length=hop_length, 
                                                    n_mels=n_mels, power=2.0,
                                                   fmin=fmin,fmax=fmax)
        mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max) 
        image=mel_spec_db[0:224,0:224]
        im = plt.imshow(image)
        colors = im.cmap(im.norm(image))
        data = colors.astype(np.float64) / np.max(colors) # normalize the data to 0 - 1
        data = 255 * data # Now scale by 255
        img = data.astype(np.uint8)
        img = img[:,:,0:3]
        plt.imshow(img)
        imagesTensor = transform(img)
        imagesTensor = Variable(imagesTensor, requires_grad=True)
        testImages = imagesTensor.unsqueeze(0)
        outputs = model(testImages)
        _, predicted = torch.max(outputs, 1)
        print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(1)))

In [ ]:

thread1 = Thread(target = infere_Class_Type, args = ())
stream = pa.open(format=pyaudio.paFloat32,
                 channels=1,
                 rate=44100,
                 output=False,
                 input=True,
                 stream_callback=callback)
def startProgram():
    thread1.start()
    stream.start_stream()
    while stream.is_active():
        time.sleep(0.25)
if __name__ == '__main__':
    print("Starting")
    startProgram()
    time.sleep(10)
    print("stopping")
    running=False
    time.sleep(1)
    stream.close()
    pa.terminate()
    thread1.join()

Starting
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
P

Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:

KeyboardInterrupt: 

Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk
Predicted:   Punk


In [1]:
running=False
thread1.join()

NameError: name 'thread1' is not defined

In [4]:
print("Starting")
time.sleep(10)
print("stop")

Starting
stop
